In [11]:
import numpy as np
import pandas as pd

In [19]:
def generate_data(num_records):

    data = []
    
    for _ in range(0, num_records):
        smokes = np.random.rand() > 0.75

        is_male = np.random.rand() > 0.5
        if is_male:
            weight = np.random.normal(loc=80, scale=5.0)
        else:
            weight = np.random.normal(loc=60, scale=3.0)

        complication_prob = 0.20    

        if smokes:
               complication_prob += 0.10

        if is_male and weight > 90:     
               complication_prob += 0.20

        if weight < 45:            
               complication_prob += 0.10                    

        notes = ""
        if complication_prob > 0.20 and np.random.rand() > 0.6:
            notes = "high risk"

        has_complication = np.random.rand() < complication_prob       

        smokes_cat = 'no'
        if smokes:
            smokes_cat = 'yes'
        
        data.append((smokes_cat, weight, notes, has_complication))
    
    return pd.DataFrame.from_records(data, columns=['smokes', 'weight', 'notes', 'has_complication'])

In [20]:
generate_data(10)

,smokes,weight,notes,has_complication
0,no,59.802151,,False
1,no,80.313039,,False
2,no,64.794924,,False
3,yes,59.432555,,False
4,yes,78.519597,,False
5,no,84.463416,,False
6,no,56.135506,,True
7,yes,63.849577,,False
8,no,60.237643,,False
9,no,71.575839,,False


In [18]:
from sklearn.preprocessing import OneHotEncoder, label_binarize, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline

from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [28]:
train_data = generate_data(2500)
test_data = generate_data(1000)

In [29]:
encode = ColumnTransformer(transformers=[
    ('numerical_features', StandardScaler(), ['weight']),
    ('categorical_features', OneHotEncoder(handle_unknown='ignore'), ['smokes']),
    ('textual_features', HashingVectorizer(ngram_range=(1, 2), n_features=10), 'notes')])

def create_mlp():
    nn = Sequential([
        Dense(8, activation='relu'), Dropout(0.3),
        Dense(4, activation='relu'),
        Dense(2, activation='softmax')])
    nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')
    return nn

pipeline = Pipeline([
    ('features', encode),
    ('learner', KerasClassifier(create_mlp, epochs=5))])

model = pipeline.fit(train_data, train_data.has_complication)
model.score(test_data, test_data.has_complication)

Epoch 1/5


/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_4/dense_12/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_4/dense_12/embedding_lookup_sparse/Reshape:0", shape=(None, 8), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_4/dense_12/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


79/79 [==============================] - 0s 628us/step - loss: 0.7282 - accuracy: 0.4920
Epoch 2/5
79/79 [==============================] - 0s 605us/step - loss: 0.6513 - accuracy: 0.7252
Epoch 3/5
79/79 [==============================] - 0s 626us/step - loss: 0.6117 - accuracy: 0.7684
Epoch 4/5
79/79 [==============================] - 0s 618us/step - loss: 0.5650 - accuracy: 0.7696
Epoch 5/5
32/32 [==============================] - 0s 575us/step - loss: 0.5370 - accuracy: 0.7730


0.7730000019073486

In [30]:
corrupted_test_data = generate_data(1000)

corrupted_test_data.loc[corrupted_test_data.sample(frac=0.2).index, 'weight'] = 0
corrupted_test_data.loc[corrupted_test_data.sample(frac=0.2).index, 'weight'] = 1000

model.score(corrupted_test_data, corrupted_test_data.has_complication)

32/32 [==============================] - 0s 590us/step - loss: 1.7762 - accuracy: 0.6490


0.6489999890327454